# INSTRUCTIONS

1. In the toolbar above, click the "▶ Run" button <strong>THREE TIMES</strong>.
2. Use the sliders to set the initial team censuses. Then click "Start Simulator" and wait a few seconds.
3. In the table, each cell is a patient. The number in the cell is the patient's length of stay.
4. To discharge a patient, click the corresponding cell.
5. To admit a patient, click "New Patient."
6. To advance to the next day, click "Next Day."

## CELL COLOR KEY
- <span style="color:blue">Blue</span>: Team census cap
- <span style="color:green">Green</span>: New patient
- <span style="color:red">Red</span>: Most recent new patient

In [3]:
#from IPython.display import HTML

#HTML('''<script>
#code_show=true;
#function code_toggle() {
# if (code_show){
# $('div.input').hide();
# } else {
# $('div.input').show();
# }
# code_show = !code_show
#}
#$( document ).ready(code_toggle);
#</script>''')

#https://stackoverflow.com/questions/27934885/how-to-hide-code-from-cells-in-ipython-notebook-visualized-with-nbviewer

import ipywidgets as widgets
from IPython.display import display, HTML

javascript_functions = {False: "hide()", True: "show()"}
button_descriptions  = {False: "Show code", True: "Hide code"}


def toggle_code(state):

    """
    Toggles the JavaScript show()/hide() function on the div.input element.
    """

    output_string = "<script>$(\"div.input\").{}</script>"
    output_args   = (javascript_functions[state],)
    output        = output_string.format(*output_args)

    display(HTML(output))


def button_action(value):

    """
    Calls the toggle_code function and updates the button description.
    """

    state = value.new

    toggle_code(state)

    value.owner.description = button_descriptions[state]


state = False
toggle_code(state)

toggle_button = widgets.ToggleButton(state, description = button_descriptions[state])
toggle_button.observe(button_action, "value")

In [16]:
import pandas as pd
import numpy as np
import ipywidgets as widgets
from ipywidgets import HBox, Layout
from IPython.display import display, clear_output, HTML
import random
import re
import time

# Global variable to track the position of the last new patient
last_new_patient_pos = None

# Global dictionary to track the number of discharged patients for each team
discharged_patients_count = {}

# Global dictionary to track the total LOS of discharged patients for each team
discharged_los_total = {}

# Global sliders
Riven_1_slider = widgets.IntSlider(value=0, min=0, max=20, step=1, description='Riven 1 [20]:')
Riven_2_slider = widgets.IntSlider(value=0, min=0, max=15, step=1, description='Riven 2 [15]:')
Riven_3_slider = widgets.IntSlider(value=0, min=0, max=15, step=1, description='Riven 3 [15]:')
Riven_4_slider = widgets.IntSlider(value=0, min=0, max=15, step=1, description='Riven 4 [15]:')
Riven_5_slider = widgets.IntSlider(value=0, min=0, max=15, step=1, description='Riven 5 [15]:')
Riven_6_slider = widgets.IntSlider(value=0, min=0, max=15, step=1, description='Riven 6 [15]:')
Riven_7_slider = widgets.IntSlider(value=0, min=0, max=20, step=1, description='Riven 7 [20]:')
Riven_8_slider = widgets.IntSlider(value=0, min=0, max=15, step=1, description='Riven 8 [15]:')
Riven_9_slider = widgets.IntSlider(value=0, min=0, max=15, step=1, description='Riven 9 [15]:')
Riven_10_slider = widgets.IntSlider(value=0, min=0, max=15, step=1, description='Riven 10 [15]:')
VICP_slider = widgets.IntSlider(value=0, min=0, max=20, step=1, description='VICP [20]:')

def create_sliders():
    # Display instructions
    header_label = widgets.HTML('<strong>Set initial team censuses:</strong>')
    display(header_label)

    # Display the sliders
    display(Riven_1_slider, Riven_2_slider, Riven_3_slider, Riven_4_slider, Riven_5_slider, Riven_6_slider, Riven_7_slider, Riven_8_slider, Riven_9_slider, Riven_10_slider, VICP_slider)

    start_simulator_button = widgets.Button(description="Start Simulator")
    start_simulator_button.on_click(on_start_simulator_clicked)
    display(start_simulator_button)

# Function to create initial censuses based on user input from sliders
def create_data_array():
    data = []
    mean = 0
    std_dev = 8 # have set this based on trial and error

    for slider in [Riven_1_slider, Riven_2_slider, Riven_3_slider, Riven_4_slider, Riven_5_slider, Riven_6_slider, Riven_7_slider, Riven_8_slider, Riven_9_slider, Riven_10_slider, VICP_slider]:
        team_data = []
        if slider.value > 0:
            for _ in range(slider.value):
                # Sample from the normal distribution, take the absolute value, and round up so that minimum value is 1
                value = np.ceil(abs(np.random.normal(mean, std_dev)))
                team_data.append(int(value))
        data.append(team_data)

    return data

# Function to be called when "Generate Table" button is clicked
def on_start_simulator_clicked(b):
    global df, initial_losac_scores, last_new_patient_pos, discharged_patients_count

    # Reset variables
    last_new_patient_pos = None
    discharged_patients_count = {}
    discharged_los_total = {}

    # Create the data array from sliders
    data_array = create_data_array()
    team_names = ['Riven 1 [20]', 'Riven 2 [15]', 'Riven 3 [15]', 'Riven 4 [15]', 'Riven 5 [15]', 'Riven 6 [15]', 'Riven 7 [20]', 'Riven 8 [15]', 'Riven 9 [15]', 'Riv 10 [15]', 'VICP [20]']

    # Find the maximum length among all teams
    max_length = max(len(team_data) for team_data in data_array)

    # Pad each team's data to ensure equal length
    padded_data = {team: team_data + [None] * (max_length - len(team_data)) for team, team_data in zip(team_names, data_array)}

    # Create the DataFrame
    df = pd.DataFrame(padded_data)

    # Initialize dataframe to have the maximum census cap as the number of rows
    max_census_cap = max([extract_census_cap(team) for team in df.columns])
    df = df.reindex(range(max_census_cap))

    # Sort each team's list in ascending order of LOS
    df = df.apply(lambda x: pd.to_numeric(x, errors='coerce').sort_values().reset_index(drop=True))

    # Calculate initial LOSAC scores
    initial_losac_scores = calculate_losac(df)

    # Call function to draw the table
    clear_output(wait=True)
    draw_table(calculate_losac(df))

# Function to extract the census cap from the column name
def extract_census_cap(column_name):
    match = re.search(r'\[(\d+)\]', column_name)
    if match:
        return int(match.group(1))
    else:
        return None

# Function to calculate LOSAC scores
def calculate_losac(dataframe):
    losac_scores = {}

    # Calculate the minimum census cap among all teams
    min_census_cap = min([extract_census_cap(team) for team in dataframe.columns])

    for team in dataframe.columns:
        census_cap = extract_census_cap(team)
        if census_cap is not None:
            team_census = dataframe[team].dropna()
            if team_census.count() < census_cap:
                median_los = team_census.median() if team_census.count() > 0 else 1
                #losac_scores[team] = (team_census.count() - (census_cap - min_census_cap)) / median_los
                losac_scores[team] = (team_census.count() / census_cap) / median_los
            else:
                losac_scores[team] = float('inf')
    return losac_scores

# Function to handle discharging patients
def discharge_patient(selected_cell):
    global df, discharged_patients_count, discharged_los_total

    # Discharge the selected patient
    team, patient = selected_cell.split(',')
    los = df.at[int(patient), team]
    df.at[int(patient), team] = pd.NA

    # Update discharged patients count and LOS total
    discharged_patients_count[team] = discharged_patients_count.get(team, 0) + 1
    discharged_los_total[team] = discharged_los_total.get(team, 0) + (los if los is not None else 0)

    # Sort and redraw the table
    df = df.apply(lambda x: pd.to_numeric(x, errors='coerce').sort_values().reset_index(drop=True))
    clear_output(wait=True)
    draw_table(calculate_losac(df))

# Function to admit a new patient
def admit_new_patient(_):
    global df, last_new_patient_pos

    # Calculate LOSAC scores with current census
    losac_scores = calculate_losac(df)

    # Filter out teams at full capacity or include all if all are at zero census
    eligible_teams = {team: losac for team, losac in losac_scores.items() if df[team].count() < extract_census_cap(team)}

    if eligible_teams:
        # Find the minimum LOSAC score
        min_losac = min(eligible_teams.values())
        tied_teams = [team for team, losac in eligible_teams.items() if losac == min_losac]

        if len(tied_teams) > 1:
            # Determine if all tied teams are at zero census
            all_zero_census = all(df[team].count() == 0 for team in tied_teams)

            # Calculate the number of open slots for each tied team
            open_slots = {team: extract_census_cap(team) - df[team].count() for team in tied_teams}
            max_open_slots = max(open_slots.values())
            teams_with_max_open_slots = [team for team, slots in open_slots.items() if slots == max_open_slots]

            # If not all teams are at zero census, calculate mean LOS for tie-breaking
            if len(teams_with_max_open_slots) > 1:
                los_means = {team: df[team].mean() for team in teams_with_max_open_slots}
                max_los_mean = max(los_means.values())

                # Account for the fact that mean LOS can't be calculated if all zero census:
                if all_zero_census:
                  teams_with_max_los_mean = teams_with_max_open_slots
                else:
                  teams_with_max_los_mean = [team for team, los in los_means.items() if los == max_los_mean]

                # Randomly select a team if there's still a tie
                if len(teams_with_max_los_mean) > 1:
                    selected_team = random.choice(teams_with_max_los_mean)
                else:
                    selected_team = teams_with_max_los_mean[0]
            else:
                selected_team = teams_with_max_open_slots[0]
        else:
            selected_team = tied_teams[0]

        # Insert new patient for the selected team
        df[selected_team] = df[selected_team].shift(1)
        df.at[0, selected_team] = 1
        last_new_patient_pos = (0, selected_team)
        df[selected_team] = pd.to_numeric(df[selected_team], errors='coerce').sort_values().reset_index(drop=True)
    else:
        clear_output(wait=True)
        display(HTML("<div style='color: red; text-align: center;'>All teams have reached their census caps. First discharge a patient.</div>"))
        time.sleep(2)  # Wait for 2 seconds

    # Redraw the table with the updated LOSAC
    clear_output(wait=True)
    draw_table(calculate_losac(df))

# Function to increment the LOS for all patients (start new day)
def increment_day(_):
    global df

    # Increment LOS by 1 for all patients
    df = df.applymap(lambda x: x + 1 if pd.notna(x) else x)

    # Sort each team's list in ascending order of LOS
    df = df.apply(lambda x: pd.to_numeric(x, errors='coerce').sort_values().reset_index(drop=True))

    # Redraw the table with updated LOSAC
    clear_output(wait=True)
    draw_table(calculate_losac(df))

# Make sure to update the create_styled_button function with the new parameters
def create_styled_button(description, pos, is_new_patient, is_green, is_blue):
    button = widgets.Button(description=description, layout=widgets.Layout(width='auto'))
    if is_new_patient:
        button.style.button_color = 'lightcoral'  # Light red color for new patient
    elif is_green:
        button.style.button_color = 'lightgreen'  # Light green for patients with LOS of 1
    elif is_blue:
        button.style.button_color = 'lightblue'   # Light blue for patients below census cap
    return button

# Function to draw the census list table
def draw_table(losac_scores):
    global df, last_new_patient_pos, discharged_patients_count, discharged_los_total

    # Create the table grid with the number of rows in the DataFrame, plus four for the header, LOSAC, discharged patients tally, and discharged LOS
    num_rows = df.shape[0]
    table = widgets.GridspecLayout(num_rows + 4, df.shape[1] + 1)

    # Add right-aligned "Team [census cap]", "LOSAC:", and "Discharged:" labels to the left of the first column header
    table[0, 0] = widgets.HTML(value="<div style='text-align: right'>Team [Cap]:</div>")
    table[1, 0] = widgets.HTML(value="<div style='text-align: right'>LOSAC:</div>")
    table[2, 0] = widgets.HTML(value="<div style='text-align: right'>D/C Count:</div>")
    table[3, 0] = widgets.HTML(value="<div style='text-align: right'>D/C LOS:</div>")

    # Add a two-line header for each column
    for j, team in enumerate(df.columns):
        table[0, j + 1] = widgets.HTML(value=f"<center>{team}</center>")
        losac_score = losac_scores[team]
        table[1, j + 1] = widgets.HTML(value=f"<center>{losac_score:.3f}</center>")
        discharged_count = discharged_patients_count.get(team, 0)
        table[2, j + 1] = widgets.HTML(value=f"<center>{discharged_count}</center>")
        total_los = discharged_los_total.get(team, 0)
        discharged_los = total_los / discharged_count if discharged_count > 0 else 0
        table[3, j + 1] = widgets.HTML(value=f"<center>{discharged_los:.2f}</center>")

    # Add right-aligned row numbers
    for i in range(num_rows):
        table[i + 4, 0] = widgets.HTML(value=f"<div style='text-align: right'>{i + 1}</div>")

    # Adjust the row index in the loop for the table cells
    for i in range(num_rows):
        for j in range(df.shape[1]):
            cell_value = df.iloc[i, j] if pd.notna(df.iloc[i, j]) else ''
            if cell_value == '':
                cell_value_str = ' '  # Display empty cells as a space
            else:
                try:
                    cell_value_str = str(int(cell_value))  # Convert non-empty cells to integers
                except ValueError:
                    cell_value_str = str(cell_value)  # Handle non-integer values gracefully
            is_new_patient = ((i, df.columns[j]) == last_new_patient_pos and cell_value == 1)
            is_green = cell_value == 1 and not is_new_patient
            team_census_cap = extract_census_cap(df.columns[j])
            is_blue = i < team_census_cap
            button = create_styled_button(cell_value_str, pos=(i, j),
                                          is_new_patient=is_new_patient, is_green=is_green, is_blue=is_blue)
            table[i + 4, j + 1] = button

            button.on_click(lambda btn, x=i, y=j: discharge_patient(f"{df.columns[y]},{x}"))

    display(table)

    # Define button style and layout
    button_style = {'font_weight': 'bold', 'font_size': '14px'}
    #button_layout = Layout(width='150px', height='50px', margin='0 25px 0 25px')  # 25px margins on left and right
    button_layout = Layout(margin='0 25px 0 25px')  # 25px margins on left and right

    # Create buttons
    new_patient_button = widgets.Button(description="New Patient", style=button_style, layout=button_layout, button_style='success')
    new_patient_button.on_click(admit_new_patient)

    next_day_button = widgets.Button(description="Next Day", style=button_style, layout=button_layout, button_style='info')
    next_day_button.on_click(increment_day)

    # Omit this button
    #reset_table_button = widgets.Button(description="Reset to Day 1", style=button_style, layout=button_layout, button_style='warning')
    #reset_table_button.on_click(reset_table)

    start_over_button = widgets.Button(description="Start Over", style=button_style, layout=button_layout, button_style='danger')
    start_over_button.on_click(start_over)

    # Arrange buttons in a horizontal layout, centered
    #buttons_layout = HBox([new_patient_button, next_day_button, reset_table_button, start_over_button],
    #                      layout=Layout(justify_content='center'))
    buttons_layout = HBox([new_patient_button, next_day_button, start_over_button],
                           layout=Layout(justify_content='center'))
    display(buttons_layout)

def reset_table(_):
    global df, initial_losac_scores, last_new_patient_pos, discharged_patients_count

    # Reset variables
    last_new_patient_pos = None
    discharged_patients_count = {}
    discharged_los_total = {}

    # Call the function as if the "Generate Table" button was clicked
    on_start_simulator_clicked(None)

def start_over(_):
    # Clear the output
    clear_output(wait=True)
    create_sliders()

# Draw the initial table
create_sliders()

#display(toggle_button)

GridspecLayout(children=(HTML(value="<div style='text-align: right'>Team [Cap]:</div>", layout=Layout(grid_are…